- With only keeping one character in the context, there were 27 possibilities.
- If we start keeping context of more previous characters, the possibilities of context keep on increasing exponentially. (27 * 27 = 729), (27 * 27 * 27 = 20k) 

- [Paper link](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
- [makemore repo link](https://github.com/karpathy/makemore)

- While we are focusing on a character level language model, the paper focuses on word level language model (17k words) but the model approach that we are going to use is going to be similar.

- Take each word and associate let's say a 30 dimensional feature vector to each word.
- Very crowded (a 30 dimensional feature vector), a lot of points for a very small space.
- Every word is embedded to a 30 dimensional space.
- Initially these words are spread out randomly.
- Then we will tune the embeddings of these words using back propagation.
- Words that are very similar to each other would be quite close to each other.
- Example, when a neural network is trained on a lot of dataset, it could come to know that cats and dogs are used in a lot of similar context and are pets. 

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [19]:
# build the dataset

block_size = 3 # how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # log added to show the context and the the corresponding next character
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

X.shape

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


torch.Size([32, 3])

# Implementing the embedding lookup table (C)
- In the paper, they had 17k words which are embedded in a space as small as 30 dimensional space which gives 17k cross 30 matrix.
- Since in our case, we are only dealing with characters and we have 27 possible characters, we will embed in an even smaller space of 2 dimensional space which gives 27 * 2 matrix.
- C acts as our first layer to the neural network. They have no non-linearity (no tanh)


In [6]:
C = torch.randn((27, 2))

In [7]:
C[5]

tensor([0.7382, 0.1996])

In [8]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([0.7382, 0.1996])

In [15]:
# C[[5, 6, 7]] we can index for multi-dimensional vectors as well.
# C[X] ==> X.shape is (32, 3). C[X].shape will (32, 3, 2) ==> a 2-dimension vector for each index.
# first layer
emb = C[X]

# Implementing the hidden layer
* The number of inputs to the neuron of this layer is going to be 3 * 2. We have a 2-dimensional space for each character.
* The no. of neurons is arbitrary, lets keep 100.
* We will have tensors for weights and biases. Ideally the layer would just do: `emb@W1 + b1`
* emb is 32, 3, 2 ==> got changed to 96, 2 and W1 is 6, 100. Hence cannot be multiplied.
* Remember that we have 3 previous characters. emb[:, 0, :] will give embeds of the first character. (3rd previous)
* We want to change 32, 3, 2 to 32, 6. We will concatenate across the dimension 1 instead of dimension 0.
* We also used torch.unbind to generalize and not requiring to do emb[:, i, :] for all i. torch.unbind returns a tuple of all slices along a dimension, after removing it.
* There is a much more efficient way to do this using torch.view.
* torch.view helps us to convert a torch from one dimension to another efficiently.
* This is efficient because internally, torch stores everything in a single dimension. We can check this via torch.storage.

In [17]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [18]:
emb @ W1 + b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [27]:
torch.cat([emb[:,0,:], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [28]:
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [36]:
W1.storage()

(torch.Size([6, 100]),
  0.10918485373258591
  -0.461626797914505
  -0.5156719088554382
  0.7642245888710022
  -0.288330078125
  0.05680552124977112
  -1.0864781141281128
  -1.0934231281280518
  0.08613672107458115
  1.3829567432403564
  -1.1412547826766968
  -0.236305370926857
  0.12988881766796112
  -0.6077059507369995
  0.1187981441617012
  -0.4598065912723541
  0.665322482585907
  -1.6671355962753296
  -0.2006147801876068
  -0.05465042218565941
  1.861762285232544
  -0.574219822883606
  0.11379670351743698
  0.23381808400154114
  0.46911364793777466
  -0.4386537969112396
  0.7614145874977112
  -0.8022424578666687
  -0.40905603766441345
  2.1134748458862305
  0.0637543648481369
  -0.30013832449913025
  0.8704205751419067
  -0.4817836880683899
  -0.46995142102241516
  0.021928006783127785
  0.948181688785553
  -0.5999742746353149
  -0.35280749201774597
  0.008993294090032578
  -0.4769386053085327
  0.2660994827747345
  0.1439337581396103
  -0.8809175491333008
  -0.45529982447624207
 